# The Annotated GPT-2 따라가기
[amaarora의 The Annotated GPT-2](https://amaarora.github.io/2020/02/18/annotatedGPT2.html?fbclid=IwAR14hZIpBSK6JY8BwAEnEybwy7gpm43HsCt4EqcBy6Tr7ZW6ve9bW9XOXuQ)을 따라가며 정리

## Introduction
GPT-2는 마법과도 같이 보인다. 그것은 빛나고 아름다워 보인다. 이 글을 통해 그 마술을 벗겨내고 싶다. 숨겨진 모든트릭들을 그때까지, 당신이 끝내다 읽는것을 이 포스트를. 이것은 나의 목표. 만들다 간단하게, 가능한한 간단히, 이해하는것을 열망하다, GPT-2 모델이 동작하는 밑바닥까지.

> 구현은 Hugging Face의 구현에서 간단한 본질을 유지했다. 만약 당신이 원한닫 학습하는것 GPT2 모델을 병렬 GPU들에 대해, 체크포인트를 저장하는것 파인튜닝하는 동안, 예측 과제를 여러 CPU에 대해 동작하는 것을, 나는 추천한다 hugging face의 API를 사용하는것을. 간단한 튜토리얼 또한 제공하고 있다.
  
새로운 것을 만드는것이 하닌 기존에 존재하는 훌륭한 자원들을 이용해서 쉽게 만든다 독자가 GPT-2를 이해하는 것을. 나는 이것을 남겨두엇따 독자가 기초를 만드는것을 그들이 선택한 어떤 영역에서.

> 훌륭한 빌딩을 약간 토대 위에 지을 수는 없다. 강력한 훌륭한 건축물을 구축하기 위해서는 견고한 토대를 가져아한다.

## Prerequisites
이 포스트는 독자들이 트랜스포머와 어텐션에 대한 단단한 이해를 가진것을 가정한다. GPT-2는 12-layer의 Transformer Decoder를 사용한다. 트랜스포머를 이해하기 위한 훌륭한 예제는 아래와 같다
    1. [The illustrated Transformer](http://jalammar.github.io/illustrated-transformer/)
    2. [The Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)
    3. [Introduction to the Transformer](https://www.youtube.com/watch?v=AFkGPmU16QA&list=PLtmWHNX-gukKocXQOkQjuVxglSDYWsSh9&index=18&t=0s)

# Language Models are Unsupervised Multitask Learners

## Abstract
NLP 태스트 QA, 기계번역, 기계독해, 요약들은 각 태스크들에 한정된 데이터 셋으로 supervised learning 접근 되었다. GPT-2는 언어모델이 어떤 명시적 감독 없이  이러한 태스크들에 학습할 수 있음을 증명한다. WebText라 불리는 수백만의 웹페이지들로 구성된 데이터 셋을 학습시킬때.  
  
GPT-2의 가장큰 언어 모델은 1.5B 파라미터를 가진 트랜스포머이다. 트랜스 포머는 8개 언어모델 데이터 셋 중 7개의 결과에 대해 SOTA를 달성했다/ 제로샷 세팅에 대해서. 그러나 WebText엔 적합하지 않았다. 모델로부터 샘플들을 이러한 개선들을 반영하고 텍스트의 일관성을 포함한다. 이러한 발견은 보장 한다/ 자연어 처리 시스템을 구축하는데/ 자연스럽게 일어난 증명으로부터 태스크를 수행하는것을 학습하는것. 

> 제로샷 세팅은 언어모델을 파인튜닝하지 않고, 바로 타겟 데이터 셋에 대해 inferences 하는것이다. 
예를들어. WebText로 Language Model을 pre-training 한 후, 바로 아마존 영화 리뷰 데이터셋에 대해 시도 및 예측한다. 

## Model Architecture(GPT-2)
GPT 언어 모델에 대해 2017 트랜스포머 구조를 사용했으며, 모델은 대체로 OpenAI의 GPT 모델에 약간 수정 후 사용했다. Layer Normalization은 각 sub-block의 input으로 이동. activation function 사용 전에 residual network를 사용. 마지막 self-attention block 이후에 layer normalization 추가. 
- vocab 사이즈는 50257개 까지 확장
- context size도 512에서 1024개 토큰 까지 확장
- 배치 사이즈는 512가 사용. 


> 위 내용이 GPT-2 논문에서 설명된 모델의 전부. GPT 모델 구조를 살펴볼 필요성

## Model Specification(GPT)
![](https://amaarora.github.io/images/gpt-architecture.PNG)
- **masked self-attention head**(768 차원, 12개 어텐션 헤더)12 layer의 transformer decoder로만 이루어진 모델사용.  
- **position-wise feed-forward networks**는 3072 차원의 state들을 사용.
- adam optimization과 2.5e-4 learning rate 사용
- 100 epoch와 랜덤 샘플링된 64 minibatch 사용.
- 40000개 BPE vocab 사용. 
- attention dropout = 0.1 for regularization/
- activation function = GELU


### Import

In [1]:
import torch
import copy
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules import ModuleList
from torch.nn.modules.normalization import LayerNorm
import numpy as np
import os
from tqdm import tqdm_notebook, trange
import logging
logging.basicConfig(level = logging.INFO)
logger = logging.getLogger()

### Transformer Decoder inside GPT-2
긴 시퀀스를 다루기 위해. GPT-2에서는 멀티헤드어텐션을 수정했다/ 메모리 사용을 줄이기 위해/ Q와 K의 Dot product를 제한함으로써.  
![](https://amaarora.github.io/images/Attention-formula.PNG)

In [2]:
class Conv1D(nn.Module):
    def __init__(self, nx, nf):
        super().__init__()
        self.nf = nf
        w = torch.empty(nx, nf)
        nn.init.normal_(w, std=0.02)
        self.weight = nn.Parameter(w)
        self.bias = nn.Parameter(torch.zeros(nf))

    def forward(self, x):
        size_out = x.size()[:-1] + (self.nf,)
        x = torch.addmm(self.bias, x.view(-1, x.size(-1)), self.weight)
        x = x.view(*size_out)
        return x

### Conv1D Layer 
CONV1D 레이어는 LINEAR 레이어 자체로 생각할 수 있다. 본질적으로, 그것은 크기 self.nf의 최종 치수를 갖기 위해 초기 텐서 x(최종 치수 x.size (-1)를 그것에 전달하고 있다.

In [7]:
d_model = 768
conv1d  = Conv1D(d_model, d_model*3)
x       = torch.rand(1,4,d_model) #represents a sequence of batch_size=1, seq_len=4 and embedding_sz=768, something like "Hello how are you"
x       = conv1d(x)
x.shape


torch.Size([1, 3, 2304])

위 예를 보면 conv1d에 의해 리턴된 결과는 최초 사이즈의 3배이다. GPT-2에서 Query, Key, Value 행렬을 캐스트 할 수 있다.  
  


In [9]:
query, key, value = x.split(d_model, dim=2)

query.shape, key.shape, value.shape 

(torch.Size([1, 3, 768]), torch.Size([1, 3, 768]), torch.Size([1, 3, 768]))

Hugging face의 weight를 사용하기 위해 conv1d를 재사용한다.


### FeedForward Layer

In [10]:
class FeedForward(nn.Module):
    def __init__(self, dropout, d_model=768, nx=768*4):
        super().__init__()
        self.c_fc    = Conv1D(d_model, nx)
        self.c_proj  = Conv1D(nx, d_model)
        self.act     = F.gelu
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        return self.dropout(self.c_proj(self.act(self.c_fc(x))))


FeedForwar 레이어는 Attention layer(768)의 output을 입력으로 사용한다. attention layer(768)는 nx(768*4)로 캐스팅된다. 그리고 GELU activation 함수와 0.1 dropout을 사용한다.

  
**GPT** 논문에서 언급
> Position-Wise-Feed-Forward 네트워크에서는 3072(768*4)차원의 inner states 사용.

### Scaled Dot-Product Attention
이때 입력으로는 **dk** 차원을 가진 Query와 Key, **dv** 차원을 가진 Value들로 구성되며, Query와 Key의 **Dot-Product** 연산 후 dk의 제곱근으로 나누어 scale한 후 softmax로 값을데 대한 가중치를 얻는다. 
![](https://amaarora.github.io/images/Attention-dot-product.PNG)
![](https://amaarora.github.io/images/Attention-formula.PNG)

#### 두가지 어텐션
일반적으로 사용하는 어텐션은 addictive와 dot-product로 나누어지며, 둘은 이론적인 복잡성에서 비슷하지만, dot-product가 더 빠르고, 메모리 효율적이다. 왜냐하면 고도로 최적화된 행렬 곱셈 코드를 사용할 수 있기 때문이다. 작은 값에 대해서 dk 두 매커니즘이 동일하게 수행되는 반면 큰 dk 값에 대해서는 dot-product 값이 더 커지며, softmax 함수를 매우 작은 gradients를 갖는 곳으로 밀어넣는다. 이런 효과를 막기위해 dk의 제곱근으로 scaling 한다.

In [12]:
class Attention(nn.Module):
    def __init__(self, d_model=768, n_head=12, n_ctx=1024, d_head=64, bias=True, scale=False):
        super().__init__()
        self.n_head  = n_head
        self.d_model = d_model
        self.c_attn  = Conv1D(d_model, d_model*3)
        self.scale   = scale
        self.softmax = nn.Softmax(dim=-1)
        self.register_buffer("bias", torch.tril(torch.ones(n_ctx, n_ctx)).view(1, 1, n_ctx, n_ctx))
        self.dropout = nn.Dropout(0.1)
        self.c_proj  = Conv1D(d_model, d_model)
        
    def split_heads(self, x):
        "return shape [`batch`, `head`, `sequence`, `features`]"
        new_shape = x.size()[:-1] + (self.n_head, x.size(-1)//self.n_head) 
        x = x.view(*new_shape)
        return x.permute(0, 2, 1, 3) 
    
    def _attn(self, q, k, v, attn_mask=None):
        scores  = torch.matmul(q, k.transpose(-2, -1))
        if self.scale: scores = scores/math.sqrt(v.size(-1))
        nd, ns  = scores.size(-2), scores.size(-1)
        if attn_mask is not None: scores = scores + attn_mask
        scores  = self.softmax(scores)
        scores  = self.dropout(scores)
        outputs = torch.matmul(scores, v)
        return outputs
    
    def merge_heads(self, x):
        x         = x.permute(0, 2, 1, 3).contiguous()
        new_shape = x.size()[:-2] + (x.size(-2)*x.size(-1),)
        return x.view(*new_shape)
        
    def forward(self, x):
        x        = self.c_attn(x) #new `x` shape - `[1,3,2304]`
        q, k, v  = x.split(self.d_model, dim=2)
        q, k, v  = self.split_heads(q), self.split_heads(k), self.split_heads(v)
        out      = self._attn(q, k, v)
        out      = self.merge_heads(out)
        out      = self.c_proj(out)
        return out

### Multi-Head Attention
단일 어텐션을 d_model(query,key,value의 차원)으로 수행하는 것 대신에. 우리는 그것이 더 유용하다는것을 알아냈다/ 선형적으로 query,key,value들을 h번 다르게 투영하는 것이. dq, dk, dv에 학습된 선형 projection. query, key, value들에 대한 이런 projection들 각각 attention을 병렬적으로 수행 가능하고 dv 차원을 가진 output value를 산출한다.  
이 값들은 concat 되어지고, 다시 final value로 project 된다.  
![](https://amaarora.github.io/images/Transformers-multi-head-attention.PNG)  
![](https://amaarora.github.io/images/multi-head-attn-formula.PNG)

현재 이 예제에서는 h=8 인 어텐션 레이어 또는 헤더를 사용하고 있고, dk= dv = d_model/h = 64  이다.  
각 헤드의 차원이 줄여졌기 때문에 전체 컴퓨팅 비용은 single-head attention과 비슷하다.

> [batch_size, seq_len, d_model]=>[1, 4, 768] 사이즈를 [batch_size, n_head, seq_len, d_model/n_head]=>[1, 12, 4, 64]로 split 한다. **split_heads**에서 분할하고, **merge_heads**에서  다시 원래 shape [batch_size, seq_len, d_model]으로 concat 한다.

## GPT-2 Model Architecture in Code
![](https://amaarora.github.io/images/gpt-architecture.PNG)  
GPT-2 모델의 구조도 Multi-Head Attention과 FeedForward 레이어를 사용하고. 12개의 Transformer의 Decoder Block으로 구성 되어있다.

![](https://amaarora.github.io/images/GPT-transformer-block.PNG)  

### Transformer Decoder Block

In [13]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model=768, n_head=12, dropout=0.1):
        super(TransformerBlock, self).__init__()
        self.attn        = Attention(d_model=768, n_head=12, d_head=64, n_ctx=1024, bias=True, scale=False)
        self.feedforward = FeedForward(dropout=0.1, d_model=768, nx=768*4)
        self.ln_1        = LayerNorm(d_model)
        self.ln_2        = LayerNorm(d_model)
                
    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.feedforward(self.ln_2(x))
        return x

## The GPT-2 구조
**GPT 논문 인용**
> GPT는 transformer decoder로만 구성된 12개의 layer를 masked self-attention head(멀티 헤드 12개, 768 차원).

GPT-2에서도 위의 TransformerBlock을 12개 사용

In [14]:
def _get_clones(module, n):
    return ModuleList([copy.deepcopy(module) for i in range(n)])

class GPT2(nn.Module):
    def __init__(self, nlayers=12, n_ctx=1024, d_model=768, vcb_sz=50257):
        super(GPT2, self).__init__()
        self.nlayers = nlayers
        block        = TransformerBlock(d_model=768, n_head=12, dropout=0.1)
        self.h       = _get_clones(block, 12)
        self.wte     = nn.Embedding(vcb_sz, d_model) 
        self.wpe     = nn.Embedding(n_ctx, d_model)
        self.drop    = nn.Dropout(0.1)
        self.ln_f    = LayerNorm(d_model)
        self.out     = nn.Linear(d_model, vcb_sz, bias=False)
        self.loss_fn = nn.CrossEntropyLoss()
        self.init_weights()
    
    def init_weights(self):
        self.out.weight = self.wte.weight
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, (nn.Linear, nn.Embedding, Conv1D)):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if isinstance(module, (nn.Linear, Conv1D)) and module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    
    def forward(self, src, labels=None, pos_ids=None):
        if pos_ids is None: pos_ids = torch.arange(0, src.size(-1)).unsqueeze(0)
        inp = self.drop((self.wte(src)+self.wpe(pos_ids)))
        for i in range(self.nlayers): inp = self.h[i](inp)
        inp     = self.ln_f(inp)
        logits  = self.out(inp)
        outputs = (logits,) + (inp,)
        
        if labels is not None:
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            loss = self.loss_fn(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            outputs = (loss,) + outputs
            return outputs
        return logits

GPT-2에서 입력은 Token Embedding과 Positional encoding의 합을 드롭아웃을 거쳐서 전달한다. 각 Transformer Decoder Block은 두 서브 파트로 나뉘며, 하나는 Attention, 다른 하나는 FeedForward 네트워크 이다.

## Language Model과 Classification
#### 1) Language Model
GPT-2를 Language Model로 사용할때, input을 마지막 LayerNorm에 전달하고,[768, vocab_size] (50257) 형태인 최종 레이어를 통과하게 되면, [1, 4, 50257] 형태의 output을 얻는다. 이때 output은 다음 단어를 나타내는 logit이고, 이것을 softmax 후 vocab 안에서 가장 높은 확률을 같는 위치인 argmax를 얻을 수 있다.  
  
#### 2) Classification
GPT-2 구조에서 Linear layer의 형태를 [768, N]으로 하고, 이때, N은 분류하고자 하는 클래스의 갯수를 의미한다. Classification을 수행하기 위해. **CrossEntropyLoss**를 사용한다. 
  
  
> 이것이 RNN과 같지 않게 positional encoding을 사용하여 입력을 병렬적으로 사용가능한 Tansformer Decoder 기반의 GPT-2의 마법에 대한 전부이며, Multi Head Attention과 FeedForward로 구성된 12개의 Decoder를 통과해 최종 출력을 얻는다.

## Hugging Face Pretrained 모델을 이용한 샘플 텍스트 생성
Pretrained 모델을 사용하기 위해 다음 경로의 gpt-2 모델을 '.'경로에 다운 받아야 한다. https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-pytorch_model.bin


In [17]:
model = GPT2()
# load pretrained_weights from hugging face
# download file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-pytorch_model.bin to `.`

model_dict = model.state_dict() #currently with random initialization
state_dict = torch.load("./gpt2-pytorch_model.bin") #pretrained weights

old_keys = []
new_keys = []
for key in state_dict.keys(): 
    if "mlp" in key: #The hugging face state dict references the feedforward network as mlp, need to replace to `feedforward` be able to reuse these weights
        new_key = key.replace("mlp", "feedforward")
        new_keys.append(new_key)
        old_keys.append(key)

for old_key, new_key in zip(old_keys, new_keys): 
    state_dict[new_key]=state_dict.pop(old_key)

pretrained_dict = {k: v for k, v in state_dict.items() if k in model_dict}

model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)
model.eval() #model in inference mode as it's now initialized with pretrained weights

GPT2(
  (h): ModuleList(
    (0): TransformerBlock(
      (attn): Attention(
        (c_attn): Conv1D()
        (softmax): Softmax(dim=-1)
        (dropout): Dropout(p=0.1, inplace=False)
        (c_proj): Conv1D()
      )
      (feedforward): FeedForward(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (1): TransformerBlock(
      (attn): Attention(
        (c_attn): Conv1D()
        (softmax): Softmax(dim=-1)
        (dropout): Dropout(p=0.1, inplace=False)
        (c_proj): Conv1D()
      )
      (feedforward): FeedForward(
        (c_fc): Conv1D()
        (c_proj): Conv1D()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (2): 

In [21]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
context   = torch.tensor([tokenizer.encode("The planet earth")])


def generate(context, ntok=20):
    for _ in range(ntok):
        out = model(context)
        logits = out[:, -1, :]
        indices_to_remove = logits < torch.topk(logits, 10)[0][..., -1, None]
        logits[indices_to_remove] = np.NINF
        next_tok = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1).squeeze(1)
        context = torch.cat([context, next_tok.unsqueeze(-1)], dim=-1)
    return context

out = generate(context, ntok=50)
print(tokenizer.decode(out[0]))

I0305 17:24:09.321796 4580994496 tokenization_utils.py:488] loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at /Users/a60058238/.cache/torch/transformers/f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
I0305 17:24:09.322589 4580994496 tokenization_utils.py:488] loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt from cache at /Users/a60058238/.cache/torch/transformers/d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda


The planet earth is full of beautiful, is beautiful, beautiful and earth-and it. of his, and it will not. of is- of It (, which) full-and of a of a- a) an the planet.




